# Assignment 8 - NMT Skeleton
### Alexandria Benedict 11/28

The aim of this homework is to familiarize you with sequence-to-sequence language modeling, specifically using an encoder-decoder model. In this notebook, you are provided with pre-written code for a simple sequence-to-sequence model that already works and learns how to reverse short sequences of numbers.

If you run this whole jupyter notebook, it will learn to reverse short sequences of numbers. Although much of this code you will not be modifying, we recommend reading through it to get a sense of how the model and training works.

This starter code is based on [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) by Sean Robertson from the PyTorch website and the COMS course at Columbia University by Professor Kathy McKeown. 

### Overview

Your assignment is to:
 1. adapt this notebook to work on the English-Italian language pair from Tataoeba website
 2. Implement a beam search function
 

Write all your code **in this jupyter notebook**. Cells are provided where you should be implementing your code. 

You do not need to modify any code to train the model. You may modify the `trainIters` function, if you would like to improve how you track progress, or change parameters while training. For example, it can be useful to decrease the teacher-forcing ratio as training progresses.


I would recommend you run this notebook as it is, first. You should be able to run it with the dummy data provided without making ANY modifications (except the cell where the data are loaded). Then, start making your changes as requested.



In [1]:
#!pip install torchtext 

In [2]:
# You may modify this cell if you like

import random
import time
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchtext
import operator
import torch.nn as nn
from queue import PriorityQueue

In [3]:
# DO NO MODIFY

# this is useful for checking if your code is successfully using the GPU

mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

C:\Users\meimei\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [4]:
# DO NOT MODIFY

SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### Load dummy number reversal dataset

In [5]:
#note that my files were stored in google drive and I was using google colab to run this notebook
#you can change this cell to provide local path to load your training and dev data

# from google.colab import drive
# drive.mount('/content/drive')

# train_path = '/content/drive/My Drive/COURSES/2020/data/train/data.txt'
# dev_path = '/content/drive/My Drive/COURSES/2020/data/dev/data.txt'



### 1. Load the data (10 points)

Load in the en-it data from http://www.manythings.org/anki/ita-eng.zip

 similar to how the dummy number reversal dataset is loaded above. That is, use the same `torchtext.data.Field` and `torchtext.data.TabularDataset` classes.

In [8]:
# WRITE YOUR CODE FOR LOADING THE ITALIAN<->ENGLISH DATA HERE
#TODO 
#create 3 TSV files, English|Italian data for train|dev|test, one sentence per line.
#you can ignore this cell when you are running the code the first time through on the dummy reversal dataset
#HINT: The file is separated by tabs (\t) - you only need the English and Italian columns
#HINT: remove the index when saving to TSV the torchtext Dataloader can confuse the index with the input.
# See below for sample output

#ita_path = './ita.txt'

#df = pd.read_csv(ita_path,sep='\t',names=['eng','it','etc'])

"""
train = df.sample(frac=.60) 
dev = df.sample(frac=.20)
test = df.sample(frac=.20)
train.to_csv('train.tsv', sep = '\t', index=False)
dev.to_csv('dev.tsv', sep = '\t', index=False)
test.to_csv('test.tsv', sep = '\t', index=False)
"""
tr_path = './itatrain.txt'
tst_path = './itatest.txt'
dv_path = './itadev.txt'

src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=tr_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dv_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_test = torchtext.data.TabularDataset(
        path=tst_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )


C:\Users\meimei\anaconda3\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
C:\Users\meimei\anaconda3\lib\site-packages\torchtext\data\example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
C:\Users\meimei\anaconda3\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further info

Have a look at the vocab and some example data points.

*If you have loaded in the data correctly, the code in the cell below should work without any modification.*

In [9]:
# You may modify this cell if you like
# Note the sample output - all words no numbers
src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

#showing output below for toy dataset

20 tokens from input vocab:
 ['<unk>', '<pad>', 'I', 'Tom', 'to', 'you', 'the', 'a', 'is', 'in', "I'm", 'was', 'of', 'have', 'You', 'that', 'do', 'be', 'for', 'He']

20 tokens from output vocab:
 ['<unk>', '<pad>', '<eos>', '<sos>', 'Tom', 'di', 'è', 'a', 'non', 'che', 'Io', 'Non', 'un', 'la', 'il', 'ha', 'per', 'in', 'sono', 'una']

num training examples: 204930

example train data:
src:
 ['Tom', "isn't", 'so', 'dependable,', 'is', 'he?']
tgt:
 ['<sos>', 'Tom', 'non', 'è', 'così', 'affidabile,', 'vero?', '<eos>']


### Model definition and training functions

In [10]:
# DO NOT MODIFY

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [11]:
# DO NOT MODIFY

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
# You may modify this cell

def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

###  Create and train a model

## Download Pre-trained Encoder-Decoder

Use the following GDrive links to download a pre-trained Encoder-Decoder models so you don't have to train it yourself:
- [Encoder](https://drive.google.com/file/d/1H27fmP9pPwP5N__ESJXOYWZ-ThIV8LBc/view?usp=sharing)
- [Decoder](https://drive.google.com/file/d/1-3opHrQzC2zBO9ZaYPkx5F-i_-f1JrmB/view?usp=sharing)

These links will give you two files `encoder.pt` and `decoder.pt` you'd want to put them in the same directory is the notebook and run the cell below. This takes away the need for you to wait for a completed training of the model using the complete dataset. The pre-trained model params are:
- For both the encoder and decoder `hidden_state` = 128
- Encoder `input_size` = 25,030
- Decoder `output_size` = 46,754

If you want a different hidden_size/on your own dataset split you have to train the model yourself because you won't be able to use the pre-trained model.

**Note** - Ensure you have the params above when you instantiate EncoderRNN and DecoderRNN for you to be able to load the params properly.


In [13]:
# You may modify this cell

hidden_size = 128
encoder1 = EncoderRNN(25030,hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, 46754).to(mydevice)

In [14]:
# Load pre-trained encoder and decoder
# Reference: https://pytorch.org/tutorials/beginner/saving_loading_models.html

encoder1.load_state_dict(torch.load(str('encoder.pt'),map_location= mydevice))
decoder1.load_state_dict(torch.load(str('decoder.pt'),map_location= mydevice))

<All keys matched successfully>

Here are some guidelines for how much training to expect. Note that these *guidelines*; they are not exact.

Only 1 epoch is needed for the number reversal dataset. This produces near-perfect results, and should take less than 5 minutes to run on a CPU.



In [15]:
# You may modify this cell
# but be sure that it prints some indication of how training is progressing
# Training is optional if you load the pre-trained Encoder-Decoder


#trainIters(encoder1, decoder1, 1, print_every=1000, teacher_forcing_ratio=0.75)

In [16]:
# DO NOT MODIFY

def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

Have a look at some generated sequences! This is the fun part.

In [17]:
# You may modify this cell
# This is to ensure all the steps above were done correctly - should see sensical output
# This selects 5 random datapoints from the training data and shows the generated sequence

for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    words = evaluate(encoder1, decoder1, seq)
    print(' '.join(words))
    print()

['You', 'need', 'to', 'become', 'more', 'active.']
<sos> del non Sei a di di <eos>

['Are', 'vampires', 'real?']
<sos> Voi Questo <eos>

["That's", "Tom's", 'house', 'with', 'the', 'red', 'roof.']
<sos> Io mai mai con un con un <eos>

["I'm", 'afraid', 'of', 'heights.']
<sos> C'è Ci di <eos>

['I', "won't", 'tolerate', 'that.']
<sos> Lei po' gioca <eos>



### Implement beam search (10 points for ITCS 4111 students, 15 points for the ITCS 5111/DSBA 6010 students)

We provide an evaluation function that performs greedy decoding on any input sequence provided. 

(ITCS 4111 students) You must write a new  function that implements beam search for a beam size = 2. 

(ITCS 5111/DSBA 6010 students) You must write a new  function that implements beam search for an arbitrary beam size. Let the beam size be an input parameter to your function. 

In greedy decoding, at each decoding step the most likely word is selected, resulting in one decoded sequence output. 

In beam search, at each decoding step the top k most
likely sequences are selected. Each of these k sequences is then used to generate the next step; the top k next words per sequence are considered (for a total of k ∗ k sequences)
and the top k sequences are selected to take to the next decoding step. At the end, you have k decoded sequence outputs.


In [18]:
# Beam Search Node adapted from: 
# https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward

In [19]:
# WRITE YOUR CODE FOR THE BEAM SEARCH HERE


#there are several implementation of beam search on the web. You can adapt those for this assignment. 
#Please provide proper credit to the original source if you are doing so
#one example is here: https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py


#you may need to add a helper function to see if your implementation of beam search works
#add that helper function below

# HINT: https://torchtext.readthedocs.io/en/latest/vocab.html - use to translate word to numbers and numbers to words

def beam_decode(decoder_hidden,beam_width=2):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    topk = 1  # how many sentence do you want to generate

    # decoding goes sentence by sentence

    # TODO: define decoder input as a 2D tensor for the SOS_TOKEN
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # Number of sentence to generate
    endnodes = []
    number_required = min((topk + 1), topk - len(endnodes))

    # starting node -  hidden vector, previous node, word id, logp, length
    node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
    nodes = PriorityQueue()

    # start the queue
    nodes.put((-node.eval(), node))
    qsize = 1

    # start beam search
    while True:
        # give up when decoding takes too long
        if qsize > 2000: break

        # fetch the best node
        score, n = nodes.get()
        decoder_input = n.wordid
        decoder_hidden = n.h
        
        # TODO: if the current node is an EOS_TOKEN and previousNode is not empty
        if n.wordid.item() == EOS_TOKEN and n.prevNode != None:
            endnodes.append((score, n))
            # if we reached maximum # of sentences required
            if len(endnodes) >= number_required:
                break
            else:
                continue

        #TODO: use the trained decoder (decoder1) to decode using the input (starting with SOS) 
        # and decoder_hidden to get the decoder output and the decoder hidden weights for the new input.
        decoder_output, decoder_hidden = decoder1(decoder_input, decoder_hidden)

        # PUT HERE REAL BEAM SEARCH OF TOP
        # HINT Reference: https://pytorch.org/docs/stable/generated/torch.topk.html
        #TODO: get get the top [beam size] most likely decoder_outputs and store their probabilities and indexes
        log_prob, indexes = torch.topk(decoder_output, beam_width)
        
        nextnodes = []

        for new_k in range(beam_width):
            decoded_t = indexes[0][new_k].view(1, -1)
            log_p = log_prob[0][new_k].item()

            node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
            score = -node.eval()
            nextnodes.append((score, node))

        # put them into queue
        for i in range(len(nextnodes)):
            score, nn = nextnodes[i]
            nodes.put((score, nn))
            # increase qsize
        qsize += len(nextnodes) - 1

    # choose nbest paths, back trace them
    if len(endnodes) == 0:
        endnodes = [nodes.get() for _ in range(topk)]

    for score, n in sorted(endnodes, key=operator.itemgetter(0)):
        utterance = []
        #TODO: convert the node id to a word
        word = output_vocab.itos[n.wordid.item()]
        #TODO: store decoded word in utterance
        utterance.append(word)
        # back trace
        while n.prevNode != None:
            #TODO: get previous node
            n = n.prevNode
            #TODO convert node id to word
            # store decoded word in utterance
            utterance.append(output_vocab.itos[n.wordid.item()])


    return utterance




In [26]:
# HINT: Refer to the evaluate function to preprocess and encode the input sentence 
# and to prepare for the decoding process
def decoder_helper(encoder,decoder,sentence,max_length=MAX_LEN,beam_width = 2):
    
    # TODO: Disable autograd through the whole method - we are only computing outputs and not updating weights.
            # Recall the PyTorch tutorial video.
    # TODO: Create an input tensor containing the id's for each word in the sentence argument
    # TODO: Initialize the encoder hidden layer
    # TODO: Store the number of words in the variable input_length
    # TODO: Loop through all the input words and encode them using the trained encoder
    # TODO: After the loop is over pass on the hidden weights from the encoder to the decoder
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        encoder_hidden = encoder.initHidden()
        input_length = input_tensor.size()[0]
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
            decoder_hidden = encoder_hidden

    return beam_decode(decoder_hidden,beam_width)
    

In [27]:
# You may modify this cell

# This selects 10 random datapoints from the training data and shows the generated sequence

for i in range(10):
    item = random.choice(data_train.examples)
    seq = item.src
    tg = item.tgt
    print(f"Source Sentence: {' '.join(seq)}")
    words = decoder_helper(encoder1,decoder1,seq,beam_width = 2)
    print(f"Predicted Sentence: {' '.join([w for w in words if w not in [SOS_TOKEN,EOS_TOKEN]])}")
    print(f"Target Sentence: {' '.join(tg)}")
    print()

Source Sentence: This is fun.
Predicted Sentence: sta Avete
Target Sentence: <sos> Questo è divertente. <eos>

Source Sentence: I was tempted.
Predicted Sentence: sono Lei
Target Sentence: <sos> Ero tentata. <eos>

Source Sentence: I went to the market.
Predicted Sentence: un da andata Io
Target Sentence: <sos> Io andai al mercato. <eos>

Source Sentence: This is Chinese food.
Predicted Sentence: ritardo. sta Avete
Target Sentence: <sos> Questo è cibo cinese. <eos>

Source Sentence: I came to Tokyo to attend a conference.
Predicted Sentence: la a a caffè. Ho
Target Sentence: <sos> Sono venuto a Tokyo per partecipare a una conferenza. <eos>

Source Sentence: She thinks of her boss as a father.
Predicted Sentence: di di di è si Sono
Target Sentence: <sos> Lei considera il suo capo come un padre. <eos>

Source Sentence: Tom nervously handed Mary the knife.
Predicted Sentence: un mai è si Tom
Target Sentence: <sos> Tom passò nervosamente il coltello a Mary. <eos>

Source Sentence: Who are 